In [1]:
import os
import re

# Adjust these paths if necessary
ITEMS_LUA_PATH = r"C:\FXServer\txData\QBCoreFramework_5784FC.base\resources\[qb]\qb-core\shared\items.lua"
IMAGES_FOLDER = r"C:\FXServer\txData\QBCoreFramework_5784FC.base\resources\[qb]\qb-inventory\html\images"

# Regex to capture the filename from lines like: image = 'weapon_bat.png',
pattern = r"image\s*=\s*'([^']+)'"

missing_images = []

# 1. Read the entire items.lua
with open(ITEMS_LUA_PATH, "r", encoding="utf-8") as f:
    content = f.read()

# 2. Find all "image = 'filename.ext'" occurrences
matches = re.findall(pattern, content)

# 3. For each image name, check if it exists in the images folder
for image_name in matches:
    image_path = os.path.join(IMAGES_FOLDER, image_name)
    if not os.path.exists(image_path):
        missing_images.append(image_name)

# 4. Print the results
print("\n=== Missing Images ===")
if missing_images:
    for img in missing_images:
        print(f" - {img}")
    print(f"\nTotal missing images: {len(missing_images)}")
else:
    print("No missing images! All good.")




=== Missing Images ===
 - placeholder.png
 - weapon_candycane
 - luxuryfinish_attachment.png
 - xtc_brick.png
 - oxy_brick.png
 - meth_brick.png
 - coke_seed.png
 - tomato_seed.png
 - crafting_table_bad.png
 - crafting_table_normal.png
 - crafting_table_good.png
 - crafting_general_station.png
 - highentropyalloyingot.png
 - weapon_dildo.png
 - explosivecharge.png
 - hydraulicspreader.png

Total missing images: 16


In [11]:
# Cell 1: Configuration and Imports
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import spearmanr
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = "Data/PriceData"
SAMPLE_SIZE = 1000
WEIGHTS = [0.75, 0.15, 0.1]  # 1d, 5d, 20d

def safe_divide(a, b):
    return np.divide(a, b, out=np.zeros_like(a), where=(b != 0) & (~np.isnan(b)))

def safe_log(x):
    return np.log(np.abs(x) + 1e-10) * np.sign(x)

In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy.stats import spearmanr
from pathlib import Path
import warnings
from tqdm import tqdm
import os
import re

warnings.filterwarnings('ignore')

# Configuration
DATA_PATH = "Data/PriceData"
SAMPLE_SIZE = 1000
WEIGHTS = [0.75, 0.15, 0.1]  # 1d, 5d, 20d

def safe_divide(a, b):
    return np.divide(a, b, out=np.zeros_like(a), where=(b != 0) & (~np.isnan(b)))

def safe_log(x):
    return np.log(np.abs(x) + 1e-10) * np.sign(x)

def calculate_rolling_oos(signal, target, window=63):
    """Rolling out-of-sample validation"""
    oos = []
    if len(signal) <= window:
        return np.nan
    for i in range(len(signal)-window):
        train = signal.iloc[i:i+window]
        test = signal.iloc[i+window]
        if len(train.dropna()) > 10 and not np.isnan(test):
            corr = spearmanr(test, target.iloc[i+window]).correlation
            oos.append(corr if not np.isnan(corr) else 0)
    return np.nanmean(oos) if oos else np.nan


import numpy as np
import pandas as pd

def create_dynamic_retracement_indicator(df, 
                                         lookback=60, 
                                         ret_quantile=0.80, 
                                         vol_quantile=0.40, 
                                         price_col='Close', 
                                         volume_col='Volume'):
    """
    Create a fully adaptive 'Dynamic Retracement Indicator' (DRI) that flags 
    contrarian signals when the recent price move is extreme, yet volume 
    remains low (relative to recent history). 
    
    Parameters
    ----------
    df : pd.DataFrame
        Must contain 'Close' and 'Volume' columns (or user-specified).
    lookback : int
        Rolling window size (in bars) to calculate adaptive thresholds.
    ret_quantile : float
        Quantile to define an "extreme" absolute return over the short window.
        E.g., 0.80 means we pick the 80th percentile of absolute returns from 
        the last 'lookback' bars as a threshold.
    vol_quantile : float
        Quantile to define "low" volume environment from the last 'lookback' bars.
        E.g., 0.40 means if today's volume is below the 40th percentile of 
        past 'lookback' volumes, it is considered "low."
    price_col : str
        The name of the column in `df` containing prices.
    volume_col : str
        The name of the column in `df` containing volumes.
    
    Returns
    -------
    df : pd.DataFrame
        A copy of the input with an added 'DRI' column. This is the *signal* 
        (positive = long, negative = short, NaN = no signal).
        
    Notes
    -----
    - We compute a short-window return (3-day by default). Adjust if desired.
    - We do NOT incorporate any future returns inside the condition 
      (to avoid data leakage). 
    - Performance measurement vs. future returns can be handled afterward 
      (e.g. using df['Close'].pct_change(5).shift(-5) or some custom target).
    """
    
    df = df.copy()
    
    # 1) Compute a short-term return (3-day by default).
    #    Feel free to parameterize short_window if you'd like.
    df['ret_3d'] = df[price_col].pct_change(3)
    
    # 2) Create a rolling "extreme move" threshold based on ABS returns:
    #    e.g., the 80th percentile of the absolute 3-day returns in the last `lookback` bars.
    #    For each row, we only look at the *past* data, so shift=1 or min_periods can help.
    rolling_abs_ret = df['ret_3d'].abs().rolling(lookback, min_periods=10)
    df['abs_ret_threshold'] = rolling_abs_ret.quantile(ret_quantile)
    
    # 3) Create a rolling "low volume" threshold from the last `lookback` bars.
    rolling_volume = df[volume_col].rolling(lookback, min_periods=10)
    df['volume_threshold'] = rolling_volume.quantile(vol_quantile)
    
    # 4) Condition for "extreme move" = abs(ret_3d) > abs_ret_threshold 
    #    AND "low volume" = Volume < volume_threshold
    #    Contrarian signal = - sign(ret_3d)
    condition = (
        (df['ret_3d'].abs() > df['abs_ret_threshold']) & 
        (df[volume_col] < df['volume_threshold'])
    )
    
    df['DRI'] = np.where(condition, -np.sign(df['ret_3d']), np.nan)
    
    # Optionally, you could scale the DRI by how extreme the move is 
    # above the threshold, for example:
    #
    #   distance_factor = (df['ret_3d'].abs() - df['abs_ret_threshold']) / df['abs_ret_threshold']
    #   df['DRI'] = np.where(condition, -np.sign(df['ret_3d']) * (1 + distance_factor.clip(lower=0)), np.nan)
    #
    # But keep it simple unless you want a magnitude-based signal.
    
    # 5) Clean up extra columns if desired:
    df.drop(['ret_3d', 'abs_ret_threshold', 'volume_threshold'], axis=1, inplace=True)
    
    return df



def create_ta_features(df):

    #use the dynamic retracement indicator 
    df = create_dynamic_retracement_indicator(df)


    # Market Regime Analysis
    try:
        volatility = df['Close'].rolling(20).std() / df['Close']
        bins = [float('-inf')] + list(np.percentile(volatility.dropna(), [33.33, 66.67])) + [float('inf')]
        df['Volatility_Regime'] = pd.cut(
            volatility,
            bins=bins,
            labels=['Low', 'Medium', 'High']
        )
    except Exception as e:
        try:
            # Fallback method 1
            df['Volatility_Regime'] = pd.qcut(
                df['Close'].rolling(20).std() / df['Close'],
                q=3,
                labels=['Low', 'Medium', 'High'],
                duplicates='drop'
            ).fillna('Medium')
        except:
            # Ultimate fallback
            df['Volatility_Regime'] = 'Medium'
    

    # Volume Profile Analysis - Fixed Version
    df['Volume_Pctl'] = (
        df['Volume'].rolling(20)
        .rank(pct=True)
        .mul(4)  # Changed from 5 to 4 since rank returns 0-1
        .add(1)  # Add 1 to get range 1-5
        .fillna(method='ffill')
        .fillna(0)  # Handle any remaining NaNs
        .round()  # Round to nearest integer
        .astype('Int64')  # Use pandas nullable integer type instead of standard int
    )

    # Block Trade Detection
    df['Block_Trade'] = np.where(
        (df['Volume'] > df['Volume'].rolling(20).mean() * 2) &
        (df['High'] - df['Low'] < df['Close'] * 0.001) &
        (df['Close'] > df['Open']),
        1,
        0
    )

    # Existing Core Features
    lookback = 14
    df['MA_14'] = df['Close'].rolling(lookback).mean()
    df['STD_14'] = df['Close'].rolling(lookback).std()
    df['Volume_Rank'] = df['Volume'].rolling(lookback).rank(pct=True)
    
    # Enhanced Price Deviation
    price_deviation = (df['Close'] - df['MA_14']) / df['STD_14']
    volume_impact = np.log1p(df['Volume'] / df['Volume'].rolling(20).mean())
    df['Enhanced_Deviation'] = price_deviation * volume_impact
    
    # Conditional Logic Components
    df['MA_3'] = df['Close'].rolling(3).mean().pct_change(3)
    
    # Recent volatility measurement
    df['Max_5D_Move'] = df['Close'].pct_change().abs().rolling(5).max()
    
    # Core Indicator Logic - Single Implementation
    df['Breakout_Anticipator'] = np.nan
    mask = (
        (price_deviation.abs() > 1) & 
        (df['Volume_Rank'] > 0.9) & 
        (df['Volume'] > 1e5)  # Absolute volume filter
    )
    df.loc[mask, 'Breakout_Anticipator'] = (
        np.sign(df['MA_3']) * 
        df['Max_5D_Move'] * 
        (1 + price_deviation.abs()/2)
    )
    
    # Mean Reversion Detector
    df['Reversion_Signal'] = np.nan
    rev_mask = (
        (price_deviation.abs() > 2) & 
        (df['Volume'] > df['Volume'].rolling(20).median())
    )
    df.loc[rev_mask, 'Reversion_Signal'] = (
        -np.sign(price_deviation) * 
        df['Max_5D_Move'] * 
        np.log1p(df['Volume']/1e6)
    )
    
    # Trend Persistence Enhancer
    df['Momentum_Thrust'] = np.nan
    thrust_mask = (
        (df['Close'] > df['High'].rolling(5).max()) &
        (df['Volume'] > df['Volume'].rolling(20).quantile(0.8))
    )
    df.loc[thrust_mask, 'Momentum_Thrust'] = (
        df['Close'].pct_change(3) * 
        (df['Volume'] / df['Volume'].rolling(20).mean())
    )
    
    # Range and CEM
    window = 14
    df['Range'] = df['High'] - df['Low']
    range_ma = df['Range'].rolling(window).mean()
    df['CEM'] = np.where(
        (df['Range'] > 1.5*range_ma) & (df['Close'] > df['Open']),
        df['Close'].pct_change(3).shift(-3),
        np.where(
            (df['Range'] > 1.5*range_ma) & (df['Close'] < df['Open']),
            -df['Close'].pct_change(3).shift(-3),
            np.nan
        )
    )
    
    # Liquidity Vacuum Indicator
    vol_ma = df['Volume'].rolling(20).mean()
    df['LVI'] = np.where(
        (df['Volume'] < 0.7*vol_ma) & 
        (df['Close'].rolling(5).std() < 0.015*df['Close']),
        (df['Close'].rolling(20).quantile(0.9) - df['Close']) / df['Close'],
        np.nan
    )
    
    # Paradoxical Reversal Signal
    ret_3d = df['Close'].pct_change(3)
    df['PRS'] = np.where(
        (ret_3d.abs() > 0.15) & 
        (df['Volume'] < df['Volume'].rolling(10).quantile(0.4)),
        -np.sign(ret_3d) * df['Close'].pct_change(5).shift(-5),
        np.nan
    )
    
    # Volume-Confirmation Divergence
    price_high = df['Close'].rolling(14).max()
    vol_high = df['Volume'].rolling(14).max()
    df['VCD'] = np.where(
        (df['Close'] == price_high) & (df['Volume'] < 0.8*vol_high),
        -df['Close'].pct_change(5).shift(-5),
        np.nan
    )
    
    # Fractal Efficiency Ratio
    move = df['Close'].diff(5).abs()
    volatility = df['High'].rolling(5).max() - df['Low'].rolling(5).min()
    df['FER'] = np.where(
        (move/volatility > 0.7) & (df['Volume'] > df['Volume'].rolling(20).mean()),
        move * np.sign(df['Close'].diff(5)) / df['Close'],
        np.nan
    )
    
    # Naive Benchmark
    df['Naive_Benchmark'] = df['Close'].pct_change()
    
    # Apply optimizations first to create VCD_Directional


    df = optimize_indicators(df)
    
    # Create SMFI after optimize_indicators but before Smart_Money_Flow
    df['SMFI'] = np.where(
        (df['Block_Trade'] == 1) &
        (df['Volume_Pctl'] >= 4),
        df['Close'].pct_change() * volume_impact,
        np.nan
    )
    
    # Now create Smart_Money_Flow after SMFI exists
    df['Smart_Money_Flow'] = (
        0.5 * df['VCD_Directional'].fillna(0) +
        0.3 * df['SMFI'].fillna(0) +
        0.2 * df['Block_Trade'].fillna(0)
    ).replace(0, np.nan)
    
    # Volatility Regime Adjusted Signals
    for regime in ['Low', 'Medium', 'High']:
        mask = df['Volatility_Regime'] == regime
        df[f'VCD_Regime_{regime}'] = np.where(
            mask,
            df['VCD_Directional'] * [0.8, 1.0, 1.5][['Low', 'Medium', 'High'].index(regime)],
            np.nan
        )
    
    # Adaptive Signal Scaling - Move this here from optimize_indicators
    # Volatility Regime Adjusted Signals
    regime_multipliers = {'Low': 0.8, 'Medium': 1.0, 'High': 1.5}
    for regime in ['Low', 'Medium', 'High']:
        df[f'VCD_Regime_{regime}'] = np.where(
            df['Volatility_Regime'] == regime,
            df['VCD_Directional'] * regime_multipliers[regime],
            np.nan
        )
    
    # Drop intermediate columns
    df = df.drop(columns=['MA_14', 'STD_14', 'Volume_Rank', 'MA_3', 'Max_5D_Move', 'Range'])

    
    return df

def optimize_indicators(df):
    # Original inverse transformations
    df['Reversion_Signal_Inverse'] = -df['Reversion_Signal']
    df['FER_Inverse'] = -df['FER']
    
    # Original indicators from previous version (moved up)
    df['VCD_Directional'] = np.where(df['VCD'] < 0, -df['VCD'], np.nan)
    
    # Enhanced VCD Directional with relaxed thresholds
    df['VCD_Directional_v2'] = np.where(
        ((df['Close'] - df['MA_14']).abs()/df['STD_14'] > 1.5) &
        (df['Volume'] > df['Volume'].rolling(20).median()),
        -np.sign(df['Close'] - df['MA_14']) * df['STD_14']/df['Close'],
        np.nan
    )
    
    # Enhanced PRS with volatility scaling
    prs_threshold = df['PRS'].quantile(0.6)
    df['PRS_Enhanced_v2'] = np.where(
        df['PRS'] > prs_threshold,
        df['PRS'] * (1 + df['Close'].pct_change().rolling(5).std()*100),
        np.nan
    )
    
    df['PRS_Enhanced'] = np.where(
        df['PRS'] > df['PRS'].quantile(0.7), 
        df['PRS'] * 1.5,
        np.nan
    )
    
    # Earnings Thrust indicators
    earnings_window = 5
    df['Earnings_Thrust'] = np.where(
        df['Volume'].rolling(earnings_window).std() > 2*df['Volume'].rolling(20).std(),
        df['Close'].pct_change(earnings_window).shift(-earnings_window) * 
        np.log1p(df['Volume']/1e6),
        np.nan
    )
    
    df['Earnings_Thrust_v2'] = np.where(
        df['Earnings_Thrust'].notna(),
        df['Earnings_Thrust'] * (1 + df['Volume'].pct_change().rolling(5).std()),
        np.nan
    )
    
    # LVI-Volatility Composite
    df['LVI_Vol_Composite'] = df['LVI'] * np.sqrt(
        df['Volume'].rolling(20).mean()/df['Volume']
    )
    
    # Breakout Anticipator Inverse with volume decay
    df['Breakout_Anticipator_Inverse'] = np.where(
        df['Breakout_Anticipator'].notna(),
        -df['Breakout_Anticipator'] * (df['Volume']/df['Volume'].rolling(20).mean()),
        np.nan
    )
    
    # Regime-Filtered Naive Benchmark
    vol_filter = df['Close'].rolling(20).std() < 0.015*df['Close']
    df['Naive_Benchmark_v2'] = np.where(
        vol_filter,
        df['Naive_Benchmark'],
        np.nan
    )
    
    # Weighted Composite Alpha
    df['Composite_Alpha'] = (
        0.4 * df['VCD_Directional_v2'].fillna(0) +
        0.3 * df['PRS_Enhanced_v2'].fillna(0) +
        0.2 * df['Earnings_Thrust_v2'].fillna(0) +
        0.1 * df['LVI_Vol_Composite'].fillna(0)
    ).replace(0, np.nan)
    
    df['LVI_PRIS_Combo'] = np.where(
        df['LVI'].notna() & df['PRS'].notna(),
        (df['LVI'] * 0.6 + df['PRS'] * 0.4) * df['Volume'].pct_change(3),
        np.nan
    )
    
    df['Gamma_Squeeze'] = np.where(
        (df['Close'].rolling(5).std() > 0.1*df['Close']) &
        (df['Volume'].rolling(5).mean() > 2*df['Volume'].rolling(20).mean()),
        df['High'].rolling(3).max() / df['Low'].rolling(3).min() - 1,
        np.nan
    )
    
    df['Inst_Flow'] = np.where(
        (df['Volume'] > 1e6) &
        (df['Close'] == df['High']) &
        (df['Volume'] > df['Open'] * 1000),
        df['Volume'].rolling(10).mean().pct_change(3),
        np.nan
    )
    
    

    return df









def calculate_metrics(feature, target, returns):
    valid_mask = feature.notna() & target.notna()
    if valid_mask.sum() < 10:
        return np.nan, np.nan
    
    # Handle categorical data
    if pd.api.types.is_categorical_dtype(feature):
        return np.nan, np.nan
    
    # Convert to float if needed
    feature_vals = pd.to_numeric(feature, errors='coerce').dropna()
    target_vals = target[valid_mask]
    
    ic = spearmanr(feature_vals, target_vals).correlation
    
    # Improved profit factor calculation
    direction = np.sign(feature_vals)
    realized_returns = returns.loc[feature_vals.index] * direction
    pf = realized_returns[realized_returns > 0].sum() / abs(realized_returns[realized_returns < 0].sum())
    
    if len(realized_returns) < 30:  # Minimum sample threshold
        return ic, np.nan
    
    gains = realized_returns[realized_returns > 0]
    losses = realized_returns[realized_returns < 0]
    
    if len(losses) == 0:
        pf = gains.sum() / 1e-6  # Regularized PF
    else:
        pf = gains.sum() / abs(losses.sum())
    
    return ic, pf




def analyze_outlier(indicator_name, results_df):
    outlier = results_df[results_df['Indicator'] == indicator_name]
    print(f"\nOutlier Diagnostic - {indicator_name}:")
    print(f"Signal Frequency: {outlier['Coverage'].values[0]:.2%}")
    print(f"Return Distribution: {outlier['IC'].values[0]:.2f} IC")
    print(f"Risk/Reward Profile: {outlier['ProfitFactor'].values[0]:.1f} PF")
    
    if outlier['Coverage'].values[0] < 0.05:
        print("⚠️ Warning: Low signal frequency - results likely unreliable")
    if outlier['ProfitFactor'].values[0] > 10:
        print("⚠️ Extreme PF suggests overfitting or small sample size")




def evaluate_indicators(df):
    results = []
    returns = df['Close'].pct_change().shift(-1)
    
    # Only evaluate numerical features
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    indicators = [c for c in numeric_cols 
                 if c not in ['Date', 'Ticker', 'Target', 'Open', 
                             'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    
    for col in indicators:
        ic, pf = calculate_metrics(df[col], df['Target'], returns)
        results.append({
            'Indicator': col,
            'IC': ic,
            'ProfitFactor': pf,
            'Coverage': df[col].notna().mean()
        })
    
    return pd.DataFrame(results)








In [ ]:

import gc # Garbage Collector

# Block 2: Simplified Analysis and Visualization
def process_single_file(file_path, weights):
    """Process one file and return its indicator metrics"""
    try:
        df = pd.read_parquet(file_path)
        ticker = file_path.stem
        
        # Calculate target
        df = calculate_target(df)
        
        # Create features
        df = create_ta_features(df)
        
        # Calculate metrics
        results = []
        returns = df['Close'].pct_change().shift(-1)
        numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
        indicators = [c for c in numeric_cols if c not in ['Target', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
        
        for col in indicators:
            ic, pf = calculate_metrics(df[col], df['Target'], returns)
            coverage = df[col].notna().mean()
            results.append({
                'Ticker': ticker,
                'Indicator': col,
                'IC': ic,
                'ProfitFactor': pf,
                'Coverage': coverage
            })
            
        return pd.DataFrame(results)
    
    except Exception as e:
        print(f"Error processing {file_path.name}: {str(e)}")
        return pd.DataFrame()

def run_full_analysis(data_path=DATA_PATH, sample_size=10, weights=WEIGHTS):
    """Process files individually with memory cleanup"""
    all_files = [f for f in Path(data_path).glob("*.parquet")]
    selected_files = np.random.choice(all_files, min(sample_size, len(all_files)), replace=False)
    
    all_results = []
    
    # Process files one by one with progress bar
    for file_path in tqdm(selected_files, desc="Processing Files"):
        file_results = process_single_file(file_path, weights)
        if not file_results.empty:
            all_results.append(file_results)
        
        # Explicit memory cleanup
        del file_results
        gc.collect()
    



    # Combine results
    results_df = pd.concat(all_results).groupby('Indicator').agg({
        'IC': 'mean',
        'ProfitFactor': 'mean',
        'Coverage': 'mean'
    }).reset_index()
    
    # Create visualization
    fig = px.scatter(results_df,
                    x='IC',
                    y='ProfitFactor',
                    text='Indicator',
                    title=f'Indicator Performance (n={len(selected_files)} files)',
                    labels={'IC': 'Information Coefficient', 'ProfitFactor': 'Profit Factor'},
                    hover_data=['Indicator', 'IC', 'ProfitFactor', 'Coverage'])
    
    # Add performance thresholds
    fig.add_shape(type="line", x0=0.15, y0=0, x1=0.15, y1=3,
                line=dict(color="red", width=2, dash="dot"))
    fig.add_shape(type="line", x0=-0.5, y0=1.2, x1=0.5, y1=1.2,
                line=dict(color="red", width=2, dash="dot"))
    
    fig.update_layout(
        xaxis_range=[-0.2, 0.5],
        yaxis_range=[0.8, 3],
        showlegend=False
    )
    
    # Print summary
    print("\nTop Performers:")
    print(results_df.sort_values('ProfitFactor', ascending=False).head(10).to_string(index=False))
    

    #show the graph 
    fig.show()


    return results_df

    


# Updated target calculation
def calculate_target(df):
    close = df['Adj Close']
    
    # Weighted returns with proper alignment
    df['Target'] = (0.75 * close.pct_change().shift(-1).fillna(0) +
                  0.20 * close.pct_change(5).shift(-5).fillna(0) +
                  0.05 * close.pct_change(20).shift(-20).fillna(0))
    return df

results_df = run_full_analysis(sample_size=5000)  # Now handles 500 files easily


In [2]:
import pandas as pd 



##read this file here Data\PriceData\AAPL\AAPL_DAILY.parquet and print the head 
df = pd.read_parquet('Data/PriceData/AAPL/AAPL_DAILY.parquet')
print(df.head())

        Date    Open    High     Low   Close       Volume  Average  BarCount  \
0 2024-02-27  180.60  183.93  179.56  183.06   44885592.0  181.837    191479   
1 2024-02-28  183.30  183.47  180.13  181.22   40521639.0  181.163    163556   
2 2024-02-29  180.89  182.57  179.53  180.63  105396081.0  180.689    266290   
3 2024-03-01  180.60  181.30  177.38  179.00   59067834.0  179.025    249051   
4 2024-03-04  179.93  179.93  173.79  174.60   63591036.0  174.882    286768   

   BID_Open  BID_High  ...  MIDPOINT_BarCount  BID_ASK_Open  BID_ASK_High  \
0    179.88    183.92  ...                 -1        181.57        189.10   
1    181.38    183.09  ...                 -1        181.53        183.31   
2    180.01    182.56  ...                 -1        180.64        182.57   
3    179.75    181.21  ...                 -1        179.23        181.58   
4    178.33    179.33  ...                 -1        175.80        180.44   

   BID_ASK_Low  BID_ASK_Close  BID_ASK_Volume  BID_ASK_A